In [74]:
def _clean_corpus(corpus):
    """
    Extracts words from a corpus and sanitises them.
    """
    
    allowed_chars = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    
    splitted = corpus.split()
    
    cleaned = ("".join(filter(lambda x: x in allowed_chars, word)) for word in splitted)
    cleaned = [x.lower() for x in cleaned]
    
    return cleaned


class StaticEncoder:

    def __init__(self, corpus, 
                 default_missing_key = -1,
                 default_missing_val = ""):

        self._decoder = dict(enumerate(set(corpus)))

        self._encoder = {v : k for k,v in self._decoder.items()}
        
        self._default_missing_key = default_missing_key
        
        self._default_missing_val = default_missing_val
    
    
    def encode(self, x):
        
        return self._encoder.get(x, self._default_missing_key)
    
    
    def decode(self, x):
        
        return self._decoder.get(x, self._default_missing_val)
    
train_corpus = """
Mary had a little lamb its fleece was white as snow;
And everywhere that Mary went, the lamb was sure to go. 
It followed her to school one day, which was against the rule;
It made the children laugh and play, to see a lamb at school.
And so the teacher turned it out, but still it lingered near,
And waited patiently about till Mary did appear.
"Why does the lamb love Mary so?" the eager children cry;
"Why, Mary loves the lamb, you know" the teacher did reply."
"""

cleaned_corpus = _clean_corpus(train_corpus)

In [75]:
corpus_encoder = StaticEncoder(cleaned_corpus)

In [43]:
from collections import Counter
from itertools import chain


def _generate_all_grams(corpus):
    """
    Creates a generator of all n_grams between 2, and len(corpus) -1
    Parameters:
        corpus ([str]) : corpus
    """
    n_gram_generator = chain(*(zip(*(corpus[j:] for j in range(i))) for i in range(2, len(corpus))))
    
    return n_gram_generator


def _create_n_gram_counter(n_gram_generator):
    """
    Calculates the unnormalised transition matrix as a counter.
    """
    
    # split n_grams to ((n-1), 1) pairs
    split_n_grams = ((x[:-1], x[-1]) for x in n_gram_generator)
    
    # count pairs
    n_gram_counter = Counter(split_n_grams)

    return n_gram_counter


def _create_probability_dict(n_gram_counter):
    """
    Calculates the transition probabilities as a dict of dicts.
    """

    prob_dict = {}

    for (head, tail), count in counter.items():
    
        if not head in prob_dict:
            prob_dict.update({head : {tail : count}})
        else:
            prob_dict[head].update({tail : count})

    ## normalise to unit probabilities
    for head, tail_counts in prob_dict.items():
        
        tail_counts = {tail : count * 1.0 / sum(tail_counts.values()) 
                           for tail, count in tail_counts.items()}
        
        prob_dict[head] = tail_counts
        
    return prob_dict 

In [61]:
class NGramPredictor:
    
    
    def __init__(self, prob_dict):
        """
        Creates an instance of the n-gram predictor with a trained
        transition matrix.
        Parameters
            prob_dict ({str : {str : float}}) : transition probabilites
        """
        
        self._prob_dict = prob_dict
        
    
    def predict(self, string):
        """
        Lists the predictions of the subsequent word based on an input string.
        """
        
        pattern = self._clean_input_string(string)
        
        probabilites = self._find_probability(pattern)
        
        self._print_probabilites(probabilites)
        

    def _print_probabilites(self, probabilites):
        """
        Prints the search results.
        Parameters:
            search_result ({str:float} or None) : probabilities of the following word.
        """
        
        if probabilites is None:
            print("No prediction available")
            
        else:
            sorted_ = sorted(probabilites.items(), key = lambda x: x[1], reverse = True)
            string_components = ["{0}: {1:.2f}".format(word, prob) for word, prob in sorted_]
            string = " ".join(string_components)
            print(string)
        
    
    def _clean_input_string(self, string):
        """
        Creates a tuple of the cleaned input string.
        """
        
        pattern = tuple(_clean_corpus(string))
    
        return pattern
    
        
    def _find_probability(self, pattern):
        """
        Tries to find the transition probabilites.
        """
        
        result = self._prob_dict.get(pattern, None)
        
        return result

In [62]:
n_gram_generator = _generate_all_grams(cleaned_corpus)
n_gram_counter = _create_n_gram_counter(n_gram_generator)
prob_dict = _create_probability_dict(n_gram_counter)


n_gram_predictor = NGramPredictor(prob_dict)

In [63]:
n_gram_predictor.predict('a')

little 0.50 lamb 0.50
